In [162]:
# Importing required Libraries
import requests
import pandas as pd
import smtplib
import csv
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


In [163]:
# Scrape job details
job_search_keyword = 'working student analytics'
url = 'https://www.stepstone.de/jobs/' + job_search_keyword.replace(" ", "")
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')


In [164]:
# Extract job details from the website
job_titles = []
job_title_elements = soup.find_all(class_='res-3yv1ty')
for element in job_title_elements:
    job_titles.append(element.get_text(strip=True))

company_names = []
company_name_elements = soup.find_all(class_='res-1ywdsmo')
for element in company_name_elements:
    company_names.append(element.get_text(strip=True))

locations = []
location_elements = soup.find_all('span', class_='res-dettfq')
for element in location_elements:
    locations.append(element.get_text(strip=True))

links = []
link_elements = soup.find_all('div', class_='res-17bliwk')
for link_element in link_elements:
    a_element = link_element.find('a', class_='res-fr75pv')
    if a_element:
        links.append(a_element['href'])


In [165]:

# Create a DataFrame from the scraped data
data = {
    'Job Title': job_titles,
    'Company Name': company_names,
    'Location': locations,
    'Link': links
}
df = pd.DataFrame(data)


In [167]:

# Save DataFrame to CSV
csv_filename = 'job_details.csv'
df.to_csv(csv_filename, index=False)



In [168]:
# Email the CSV file
sender_email = '' # enter your email
sender_password = '' # enter your password
receiver_email = '' # enter your password

subject = 'Daily Job Details'
body = 'Please find attached the job details for today.'



In [169]:
# Create the multipart message
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_email
message['Subject'] = subject

message.attach(MIMEText(body, 'plain'))



In [170]:
# Attach the CSV file
with open(csv_filename, 'rb') as file:
    attachment = MIMEBase('application', 'octet-stream')
    attachment.set_payload(file.read())

encoders.encode_base64(attachment)
attachment.add_header('Content-Disposition', f'attachment; filename= {csv_filename}')
message.attach(attachment)



In [174]:
# Establish a secure connection with the SMTP server (Outlook)
server = smtplib.SMTP('smtp.office365.com', 587) # code is written for an outlook mail , you can change it for gmail 
server.starttls()
server.login(sender_email, sender_password)



(235, b'2.7.0 Authentication successful')

In [175]:
# Send the email
server.send_message(message)

# Close the SMTP server
server.quit()

(221, b'2.0.0 Service closing transmission channel')